This notebook is a basic tutorial that demonstrates how to make an actor prefab and put it into a simulation using Concordia.

<a href="https://colab.research.google.com/github/google-deepmind/concordia/blob/main/examples/actor_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Colab-specific setup (use a CodeSpace to avoid the need for this).
try:
  %env COLAB_RELEASE_TAG
except:
  pass  # Not running in colab.
else:
  %pip install --ignore-requires-python --requirement 'https://raw.githubusercontent.com/google-deepmind/concordia/main/examples/requirements.in' 'git+https://github.com/google-deepmind/concordia.git#egg=gdm-concordia'
  %pip list

In [ ]:
# @title Imports

from collections.abc import Mapping
import dataclasses
import numpy as np
from IPython import display

import sentence_transformers

from concordia.agents import entity_agent_with_logging
from concordia.associative_memory import basic_associative_memory
from concordia.components import agent as agent_components
from concordia.contrib.components.agent import situation_representation_via_narrative
from concordia.document import interactive_document

from concordia.language_model import language_model
from concordia.language_model import gpt_model
from concordia.language_model import together_ai
from concordia.language_model import no_language_model

import concordia.prefabs.entity as entity_prefabs
import concordia.prefabs.game_master as game_master_prefabs

from concordia.prefabs.simulation import generic as simulation
from concordia.typing import prefab as prefab_lib
from concordia.utils import helper_functions



In [ ]:
# @title Language Model Selection: provide key or select DISABLE_LANGUAGE_MODEL

# By default this colab uses either OpenAI or TogetherAI models, so you must
# provide an API key. TogetherAI offers open models from all sources.
# Note that it is also possible to use local models or other API models,
# simply replace this cell with the correct initialization for the model
# you want to use.
API_KEY = '' #@param {type: 'string'}
GPT_MODEL_NAME = None # e.g. 'gpt-4.1-nano'
TOGETHER_MODEL_NAME = 'openai/gpt-oss-120b'
# To debug without spending money on API calls, set DISABLE_LANGUAGE_MODEL=True
DISABLE_LANGUAGE_MODEL = False

if not DISABLE_LANGUAGE_MODEL and not API_KEY:
  raise ValueError('API_KEY is required.')

model = None
if not DISABLE_LANGUAGE_MODEL:
  if GPT_MODEL_NAME and TOGETHER_MODEL_NAME:
    raise ValueError(
        'Only one of GPT_MODEL_NAME or TOGETHER_MODEL_NAME should be set.')
  if GPT_MODEL_NAME:
    model = gpt_model.GptLanguageModel(api_key=API_KEY,
                                       model_name=GPT_MODEL_NAME)
  if TOGETHER_MODEL_NAME:
    model = together_ai.OpenWeightsOpenAI(api_key=API_KEY,
                              model_name=TOGETHER_MODEL_NAME)
else:
  model = no_language_model.NoLanguageModel()

In [ ]:
# @title Setup sentence encoder

if DISABLE_LANGUAGE_MODEL:
  embedder = lambda _: np.ones(3)
else:
  st_model = sentence_transformers.SentenceTransformer(
      'sentence-transformers/all-mpnet-base-v2')
  embedder = lambda x: st_model.encode(x, show_progress_bar=False)

In [ ]:
test = model.sample_text(
    prompt=('Is moral progress like getting '
            'a progressively clearer picture of something true and deep?'),
    max_tokens=5000)
print(test)

In [ ]:
test = model.sample_choice(
    prompt=('For Richard Rorty, is moral progress like getting '
            'a progressively clearer picture of something true and deep?'),
    responses=('yes', 'no'))
print(test)

In [ ]:
# @title Load prefabs from packages to make the specific palette to use here.

prefabs = {
    **helper_functions.get_package_classes(entity_prefabs),
    **helper_functions.get_package_classes(game_master_prefabs),
}

In [ ]:
#@title Print menu of prefabs

display.display(
    display.Markdown(helper_functions.print_pretty_prefabs(prefabs)))

In [ ]:
DEFAULT_INSTRUCTIONS_COMPONENT_KEY = 'Instructions'
DEFAULT_INSTRUCTIONS_PRE_ACT_LABEL = '\nInstructions'


@dataclasses.dataclass
class MyAgent(prefab_lib.Prefab):
  """A prefab implementing an entity with a minimal set of components."""

  description: str = (
      'An entity that has a minimal set of components and is configurable by'
      ' the user. The initial set of components manage memory, observations,'
      ' and instructions. If goal is specified, the entity will have a goal '
      'constant component.'
  )
  params: Mapping[str, str] = dataclasses.field(
      default_factory=lambda: {
          'name': 'Alice',
      }
  )

  def build(
      self,
      model: language_model.LanguageModel,
      memory_bank: basic_associative_memory.AssociativeMemoryBank,
  ) -> entity_agent_with_logging.EntityAgentWithLogging:
    """Build an agent.

    Args:
      model: The language model to use.
      memory_bank: The agent's memory_bank object.

    Returns:
      An entity.
    """

    agent_name = self.params.get('name', 'Bob')

    instructions = agent_components.instructions.Instructions(
        agent_name=agent_name,
        pre_act_label=DEFAULT_INSTRUCTIONS_PRE_ACT_LABEL,
    )

    observation_to_memory = agent_components.observation.ObservationToMemory()

    observation_label = '\nObservation'
    observation = agent_components.observation.LastNObservations(
        history_length=100, pre_act_label=observation_label
    )

    situation_representation_key = 'situation_representation'
    situation_representation = (
        situation_representation_via_narrative.SituationRepresentation(
            model=model,
            observation_component_key=(
                agent_components.observation.DEFAULT_OBSERVATION_COMPONENT_KEY
            ),
            declare_entity_as_protagonist=True,
        )
    )

    guiding_principle_key = 'principle'
    principle = agent_components.question_of_recent_memories.QuestionOfRecentMemories(
        model=model,
        pre_act_label=f'{agent_name} main guiding principle:',
        question=(
            f'How can {agent_name} exploit the situation for personal '
            'gain and gratification?'
        ),
        answer_prefix=f'{agent_name} understands that ',
        add_to_memory=False,
        components=[
            DEFAULT_INSTRUCTIONS_COMPONENT_KEY,
            situation_representation_key,
        ],
    )

    components_of_agent = {
        DEFAULT_INSTRUCTIONS_COMPONENT_KEY: instructions,
        'observation_to_memory': observation_to_memory,
        agent_components.observation.DEFAULT_OBSERVATION_COMPONENT_KEY: (
            observation
        ),
        agent_components.memory.DEFAULT_MEMORY_COMPONENT_KEY: (
            agent_components.memory.AssociativeMemory(memory_bank=memory_bank)
        ),
        'situation_representation': situation_representation,
        guiding_principle_key: principle,
    }

    component_order = list(components_of_agent.keys())

    act_component = agent_components.concat_act_component.ConcatActComponent(
        model=model,
        component_order=component_order,
    )

    agent = entity_agent_with_logging.EntityAgentWithLogging(
        agent_name=agent_name,
        act_component=act_component,
        context_components=components_of_agent,
    )

    return agent

In [ ]:
prefabs['myagent__Entity'] = MyAgent()

In [ ]:
# @title Generate initial conditions for the simulation

YEAR = 1546
PLACE = 'St Andrews, Scotland'
CONTEXT = 'possible sources of conflict or drama'
NUM_STATEMENTS = 20
NAMES_TO_GENERATE = 10

prompt = interactive_document.InteractiveDocument(model)
unparsed_statements = prompt.open_question(
    question=(f"Generate a string of {NUM_STATEMENTS} facts about {PLACE} in "
              f"the year {YEAR}. Pay special attention to {CONTEXT}. Write "
              "in present tense. Separate generated facts "
              "with ' *** '."),
    max_tokens=4500,
    terminators=(),
)
statements = unparsed_statements.split('***')
statements = [s.strip() for s in statements]

for statement in statements:
  print(statement)

unparsed_names = prompt.open_question(
    question=(
        f"Generate a string of {NAMES_TO_GENERATE} names appropriate for this "
        "time and place. Include surnames. Separate them with ' *** '"),
    max_tokens=4500,
    terminators=(),
)
names = unparsed_names.split('***')
names = [n.strip() for n in names]

PLAYER_ONE = names[0]
PLAYER_TWO = names[1]

print('\n')
print(f'Player one: {PLAYER_ONE}')
print(f'Player two: {PLAYER_TWO}')

prefix = f'{PLAYER_ONE} and {PLAYER_TWO} '
premise = prompt.open_question(
    question=(f'Given the setting, why are {PLAYER_ONE} and {PLAYER_TWO} about '
              'to interact?'),
    answer_prefix=prefix,
)
premise = f'{prefix}{premise}'

print('\n')
print(premise)

player_one_context = prompt.open_question(
    question=(f'{PLAYER_ONE} has a goal or interest that, if pursued, '
              f'would complicate things for {PLAYER_TWO}. What is it?'),
    max_tokens=1000,
)

print('\n')
print(player_one_context)

player_two_context = prompt.open_question(
    question=(f'{PLAYER_TWO} has a goal or interest that, if pursued, '
              f'would complicate things for {PLAYER_ONE}. What is it?'),
    max_tokens=1000,
)

print('\n')
print(player_two_context)


In [ ]:
# @title Configure instances.

instances = [
    prefab_lib.InstanceConfig(
        prefab='basic__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': PLAYER_ONE,
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='myagent__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': PLAYER_TWO,
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='generic__GameMaster',
        role=prefab_lib.Role.GAME_MASTER,
        params={
            'name': 'default rules',
            'acting_order': 'game_master_choice',
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='dialogic__GameMaster',
        role=prefab_lib.Role.GAME_MASTER,
        params={
            'name': 'conversation rules',
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='formative_memories_initializer__GameMaster',
        role=prefab_lib.Role.INITIALIZER,
        params={
            'name': 'initial setup rules',
            'next_game_master_name': 'default rules',
            'shared_memories': statements,
            'player_specific_context': {PLAYER_ONE : player_one_context,
                                        PLAYER_TWO : player_two_context},
        },
    ),
]

In [ ]:
config = prefab_lib.Config(
    default_premise=premise,
    default_max_steps=5,
    prefabs=prefabs,
    instances=instances,
)

# The simulation

In [ ]:
# @title Initialize the simulation
runnable_simulation = simulation.Simulation(
    config=config,
    model=model,
    embedder=embedder,
)

In [ ]:
# @title Run the simulation
raw_log = []
results_log = runnable_simulation.play(max_steps=5,
                                       raw_log=raw_log)

In [ ]:
# @title Display the log
display.HTML(results_log)

```
Copyright 2024 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```